In [ ]:
!pip install wfdb
import wfdb
import matplotlib.pyplot as plt
import wfdb.processing
import pandas as pd
!pip install matplotlib==3.1.3
import numpy as np 

In [ ]:
# Downloading database
name_column = 's0010_re'
q_onset_column = 1340
t_end_column = 1756
df = pd.read_excel('/content/qt.xlsx')
list_of_ecg = [name_column]
previous_patient = 'patient001'
for index, name in enumerate(df[name_column]):
  if df.patient1[index] == 'patient285':
    continue
  print(previous_patient)
  if not pd.isna(df.patient1[index]):
    num = str(df.patient1[index][7:])
    if len(num) == 1:
      num = f'00{num}'
    elif len(num) == 2:
      num = f'0{num}'
    previous_patient = f'{df.patient1[index][:-int(len(str(df.patient1[index][7:])))]}{num}'
  list_of_ecg.append(wfdb.rdrecord(name, sampto=int(df[t_end_column][index]) + 5000, 
                                     pn_dir = f'ptbdb/{previous_patient}',
                                     channels=[0]))
list_of_ecg[0] = wfdb.rdrecord(list_of_ecg[0], sampto=int(t_end_column) + 5000, 
                                     pn_dir = 'ptbdb/patient001',
                                     channels=[0])

In [4]:
# Preparing data
list_of_q_onset = df[q_onset_column]
q_onset = [int(q_onset_column)]
list_of_t_end = df[t_end_column]
t_end = [int(t_end_column)]
for i in range(len(list_of_q_onset)):
  if list_of_q_onset[i] == 0:
    continue
  else:
    q_onset.append(list_of_q_onset[i])
    t_end.append(list_of_t_end[i])

In [5]:
!pip install scipy
from scipy import signal
from scipy.signal import butter, iirnotch, lfilter
from scipy.signal import butter, sosfilt, sosfilt_zi, sosfiltfilt, lfilter, lfilter_zi, filtfilt, sosfreqz, resample

In [6]:
# Functions to remove noises
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], analog=False, btype="band", output="sos")
    return sos


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sosfilt(sos,
                data)  
    return y


def butter_bandpass_filter_once(data, lowcut, highcut, fs, order=5):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    zi = sosfilt_zi(sos)
    z, _ = sosfilt(sos, data, zi=zi * data[0])
    return sos, z, zi


def butter_bandpass_filter_again(sos, z, zi):
    z2, _ = sosfilt(sos, z, zi=zi * z[0])
    return z2


def butter_bandpass_forward_backward_filter(data, lowcut, highcut, fs, order=5):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sosfiltfilt(sos,data, axis = 0) 
    return y

In [7]:
# Removing noises
lowcut = 1 
highcut = 20
copy_ecg = list_of_ecg.copy()
for i in range(len(copy_ecg)):
  fs = list_of_ecg[i].fs
  copy_ecg[i] = butter_bandpass_forward_backward_filter(copy_ecg[i].p_signal, lowcut, highcut, fs, order=4)

In [9]:
# Dividing the ECG graph into two sizes 100 and 600
splitted = copy_ecg.copy()
q_split = []
t_split = []
q0 = []
t0 = []
for i in range(len(splitted)):
  min_bpm = 20
  max_bpm = 230
  search_radius = int(60 * list_of_ecg[i].fs / max_bpm)
  qrs_inds = wfdb.processing.qrs.gqrs_detect(sig=splitted[i], fs=list_of_ecg[i].fs)
  peaks = wfdb.processing.correct_peaks(splitted[i].flatten(), peak_inds=qrs_inds,
                      search_radius=search_radius,
                     smooth_window_size = 150, peak_dir='up')
  left_slice = 0
  right_slice = 0
  if len(peaks) == 0 or peaks[0] > t_end[i] + 1000:
    continue
  p = 0
  for indexes in range(len(peaks)):
    if peaks[indexes] > q_onset[i] and len(peaks) > 1:
        left_slice = peaks[indexes] - 100
        right_slice = peaks[indexes] + 600
        break
  q_split.append(splitted[i][left_slice:peaks[indexes]])
  t_split.append(splitted[i][peaks[indexes]:right_slice])
  q0.append(q_onset[i] - left_slice)
  t0.append(t_end[i] - peaks[indexes])

In [10]:
# Removing inappropriate ECGs
new_t_split = []
new_q_split = []
new_q = []
new_t = []
for i in range(len(q_split)):
  if t0[i] > len(t_split) or t0[i] < 0 or q0[i] < 0:
    continue
  new_t_split.append(t_split[i])
  new_q_split.append(q_split[i])
  new_q.append(q0[i])
  new_t.append(t0[i])

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
!pip install xgboost

In [12]:
# Fitting and MAE for Q onset by XGBoost
for i in range(len(new_q_split)):
  new_q_split[i] = new_q_split[i].flatten()
train_x, test_x, train_y, test_y = train_test_split(new_q_split, new_q, test_size=0.10, random_state=12)
from xgboost import XGBRegressor
md = XGBRegressor(n_estimators=100, max_depth=3, eta=0.1, subsample=0.8, colsample_bytree=0.9, random_state=1024)
md.fit(train_x, train_y)
y_pred = md.predict(test_x)
mean_absolute_error(y_pred, test_y)

[13:38:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


5.372880150290096

In [13]:
# Fitting and MAE for T end by XGBoost
for i in range(len(new_t_split)):
  new_t_split[i] = new_t_split[i].flatten()
train_x_t, test_x_t, train_y_t, test_y_t = train_test_split(new_t_split, new_t, test_size=0.10, random_state=12)
md1 = XGBRegressor(n_estimators=1000, max_depth=3, eta=0.1, subsample=0.7, colsample_bytree=0.9, random_state=12)
md1.fit(train_x_t, train_y_t)
y_pred_t = md1.predict(test_x_t)
mean_absolute_error(test_y_t, y_pred_t)

[13:38:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


14.72509765625

In [ ]:
# Cross-validation
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
scores = cross_val_score(md, new_q_split, new_q, scoring='neg_mean_absolute_error', cv=cv)
scores1 = cross_val_score(md1, new_t_split, new_t, scoring='neg_mean_absolute_error', cv=cv)

In [18]:
# Metric value for Q with cross-validation
absolute(scores).mean()

5.4914148393518785

In [20]:
# Metric value for Q with cross-validation
absolute(scores1).mean()

18.74728504973767

In [ ]:
# Plot some ecgs to see the result of predictions 
for i in range(20):
  fig, ax = plt.subplots()
  ax.plot(list(test_x[i]) + list(test_x_t[i]))
  ax.axvline(y_pred[i], color='r')
  ax.axvline(y_pred_t[i] + 100, color='r')
  ax.axvline(test_y_t[i] + 100, color='g')
  ax.axvline(test_y[i], color='g')

In [19]:
# MAE of QT interval 
interval_length = [0.] * len(y_pred)
annotated_interval_length = [0.] * len(y_pred)
for i in range(len(interval_length)):
  interval_length[i] = y_pred_t[i] - y_pred[i]
  annotated_interval_length[i] = test_y_t[i] - test_y[i]
mean_absolute_error(interval_length, annotated_interval_length)

16.83188045726103

In [ ]:
# Measurements of metrics for different models for Q onset
model = Ridge(100, random_state=42)
model1 = Lasso(1, random_state=42)
model2 = GradientBoostingRegressor(max_depth=5, criterion="absolute_error", random_state=42)
model3 = GradientBoostingRegressor(max_depth=4, criterion="absolute_error", random_state=42)
model4 = GradientBoostingRegressor(max_depth=3, criterion="absolute_error", random_state=42)
model5 = GradientBoostingRegressor(max_depth=2, criterion="absolute_error", random_state=42)
model6 = RandomForestRegressor(max_depth=5, criterion="absolute_error", random_state=42)
model7 = RandomForestRegressor(max_depth=4, criterion="absolute_error", random_state=42)
model8 = RandomForestRegressor(max_depth=3, criterion="absolute_error", random_state=42)
model9 = RandomForestRegressor(max_depth=2, criterion="absolute_error", random_state=42)
Mmodel2 = GradientBoostingRegressor(max_depth=5,  random_state=42)
Mmodel3 = GradientBoostingRegressor(max_depth=4,  random_state=42)
Mmodel4 = GradientBoostingRegressor(max_depth=3,  random_state=42)
Mmodel5 = GradientBoostingRegressor(max_depth=2,  random_state=42)
Mmodel6 = RandomForestRegressor(max_depth=5,  random_state=42)
Mmodel7 = RandomForestRegressor(max_depth=4,  random_state=42)
Mmodel8 = RandomForestRegressor(max_depth=3, random_state=42)
Mmodel9 = RandomForestRegressor(max_depth=2, random_state=42)
lst_of_models = [model, model1, model2, model3, model4, model5, model6, model7, model8, model9,
                Mmodel2, Mmodel3, Mmodel4, Mmodel5, Mmodel6, Mmodel7, Mmodel8, Mmodel9]
lst_of_deviation = [0.] * 18
for x, i in enumerate(lst_of_models):
  i.fit(train_x, train_y)
  pr = i.predict(test_x)
  lst_of_deviation[x] = mean_absolute_error(test_y, pr)

In [ ]:
lst_of_titles = [ 
                 "Ridge(100)",
                 "Lasso(1)",
                 "GradientBoostingRegressor(max_depth=5, absolute error)",
                 "GradientBoostingRegressor(max_depth=4, absolute error)",
                 "GradientBoostingRegressor(max_depth=3, absolute error)",
                 "GradientBoostingRegressor(max_depth=2, absolute error)",
                 "RandomForestRegressor(max_depth=5, absolute error)",
                 "RandomForestRegressor(max_depth=4, absolute error)",
                 "RandomForestRegressor(max_depth=3, absolute error)",
                 "RandomForestRegressor(max_depth=2, absolute error)"
                 "GradientBoostingRegressor(max_depth=5)",
                 "GradientBoostingRegressor(max_depth=4)",
                 "GradientBoostingRegressor(max_depth=3)",
                 "GradientBoostingRegressor(max_depth=2)",
                 "RandomForestRegressor(max_depth=5)",
                 "RandomForestRegressor(max_depth=4)",
                 "RandomForestRegressor(max_depth=3)",
                 "RandomForestRegressor(max_depth=2)"
]

In [ ]:
for i in range(len(lst_of_titles)):
  print(f"{lst_of_titles[i]} MAE: {lst_of_deviation[i]}")

In [ ]:
# Measurements of metrics for different models for T onset
lst_of_deviation = [0.] * 18
for x, i in enumerate(lst_of_models):
  i.fit(train_x_t, train_y_t)
  pr = i.predict(test_x_t)
  lst_of_deviation[x] = mean_absolute_error(test_y_t, pr)

In [ ]:
for i in range(len(lst_of_titles)):
  print(f"{lst_of_titles[i]} MAE: {lst_of_deviation[i]}")